<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularized Autoencoder Implementation
This notebook demonstrates the implementation of a Regularized Autoencoder (RAE) using backpropagation, inspired by the methodologies described in "The Neural Coding Framework for Learning Generative Models".



## Dataset Loading and Preprocessing:
All the four datasets used in the paper (MNIST, KMNIST, FMNIST, CalTech101)

# 1.Preprocessing the datasets

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Normalize and convert to binary
def preprocess_dataset(x_train, x_test):
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = (x_train > 0.5).astype('float32')
    x_test = (x_test > 0.5).astype('float32')

    x_train = x_train.reshape((x_train.shape[0], 28, 28))
    x_test = x_test.reshape((x_test.shape[0], 28, 28))
    return x_train, x_test

In [3]:
# load mnist dataset as it's splitted into a training and testing sets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# 2. Loading KMNIST dataset

In [4]:
# Unlike MNIST datasets KMNIST datasets should be loaded in a little bit different way
(x_train_kmnist, y_test_kmnist), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)
def convert_to_numpy(ds):
    images, labels = [], []
    for img, lbl in tfds.as_numpy(ds):
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

# Convert datasets to numpy arrays
x_train_kmnist, y_train_kmnist = convert_to_numpy(x_train_kmnist)
x_test_kmnist, y_test_kmnist = convert_to_numpy(y_test_kmnist)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.6BPA4O_3.0.1/kmnist-train.tfrecord*...:   0%|          |…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/kmnist/incomplete.6BPA4O_3.0.1/kmnist-test.tfrecord*...:   0%|          | …

Dataset kmnist downloaded and prepared to /root/tensorflow_datasets/kmnist/3.0.1. Subsequent calls will reuse this data.


# 3. Loading FMNIST dataset

In [ ]:
# Loading the FMNIST dataset
(x_train_fmnist, y_test_fmnist), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

x_train_fmnist, y_train_fmnist = convert_to_numpy(x_train_fmnist)
x_test_fmnist, y_test_fmnist = convert_to_numpy(y_test_fmnist)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/fashion_mnist/incomplete.E3D8PW_3.0.1/fashion_mnist-train.tfrecord*...:   …

Generating test examples...: 0 examples [00:00, ? examples/s]

# Preprocessing for MNIST, KMNIST, FMNIST datasets

In [ ]:
# Preprocessing the MNIST dataset
x_train_mnist, x_test_mnist = preprocess_dataset(x_train_mnist, x_test_mnist)
# Preprocessing the KMNIST dataset
x_train_kmnist, x_test_kmnist = preprocess_dataset(x_train_kmnist, x_test_kmnist)
# Preprocessing the FMNIST dataset
x_train_fmnist, x_test_fmnist = preprocess_dataset(x_train_fmnist, x_test_fmnist)

print("Mnist train shape:", x_train_mnist.shape)
print("Mnist test shape:", x_test_mnist.shape)
print("Kmnist train shape:", x_train_kmnist.shape)
print("Kmnist test shape:", x_test_kmnist.shape)
print("Fmnist train shape:", x_train_fmnist.shape)
print("Fmnist test shape:", x_test_fmnist.shape)

# 4. Loading and Preprocessing CalTech101

In [ ]:
# Preprocess images: normalize, resize, and threshold to binary
def preprocess_image(image, label, image_size=(16, 16)):
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize
    image = tf.image.resize(image, image_size)  # Resize
    image = tf.cast(image > 0.5, tf.float32)  # Threshold to binary
    return image, label

# Load CalTech101 dataset
(ds_train, ds_test), ds_info = tfds.load(
    'caltech101',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Apply preprocessing to the dataset
ds_train = ds_train.map(lambda img, lbl: preprocess_image(img, lbl))
ds_test = ds_test.map(lambda img, lbl: preprocess_image(img, lbl))

# Convert to numpy arrays
def convert_to_numpy(ds):
    images, labels = [], []
    for img, lbl in tfds.as_numpy(ds):
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

x_train_caltech, y_train_caltech = convert_to_numpy(ds_train)
x_test_caltech, y_test_caltech = convert_to_numpy(ds_test)

# Flatten the images for the autoencoder input
x_train_caltech = x_train_caltech.reshape((x_train_caltech.shape[0], -1))
x_test_caltech = x_test_caltech.reshape((x_test_caltech.shape[0], -1))

print("CalTech101 train shape:", x_train_caltech.shape)
print("CalTech101 test shape:", x_test_caltech.shape)

In [ ]:
class RegularizedAutoencoder(tf.keras.Model):
    def __init__(self, input_shape):
        super(RegularizedAutoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=input_shape),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(20, activation='relu')
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Dense(np.prod(input_shape), activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-5)),
            tf.keras.layers.Reshape(input_shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# MSE calculation
def masked_mse(y_true, y_pred, mask):
    return np.mean(((y_true - y_pred) * (1 - mask)) ** 2)

# MNIST training

In [ ]:
# Instantiate and compile the model
mnist_input_shape = x_train_mnist.shape[1:]
mnist_autoencoder = RegularizedAutoencoder(mnist_input_shape)
mnist_autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
history = mnist_autoencoder.fit(x_train_mnist, x_train_mnist, epochs=50, batch_size=200, validation_data=(x_test_mnist, x_test_mnist))

# Evaluate the model
mnist_bce = mnist_autoencoder.evaluate(x_test_mnist, x_test_mnist)
print(f'MNIST Binary Cross-Entropy (BCE): {mnist_bce}')

# Create a mask and calculate M-MSE
MNIST_mask = np.random.choice([0, 1], size=x_test_mnist.shape, p=[0.5, 0.5])
MNIST_mse = masked_mse(x_test_mnist, mnist_autoencoder.predict(x_test_mnist), MNIST_mask)
print(f'MNIST Masked Mean Squared Error (M-MSE): {MNIST_mse}')


In [ ]:
# Get the latent representations from the encoder
mnist_latent_train = mnist_autoencoder.encoder.predict(x_train_mnist)
mnist_latent_test = mnist_autoencoder.encoder.predict(x_test_mnist)

# Train a logistic regression classifier
classifier = LogisticRegression(max_iter=500)
classifier.fit(mnist_latent_train, y_train_mnist)

# Predict on the test set
y_pred = classifier.predict(mnist_latent_test)

# Calculate classification error
classification_error = 1 - accuracy_score(y_test_mnist, y_pred)
print(f'MNIST Classification Error: {classification_error}')